In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
import os
import torch  # Check for GPU availability
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Check for GPU (if using PyTorch or compatible libraries later)
# =============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# =============================
# Step 1: Load and Explore Data
# =============================

df = pd.read_csv("/content/online_retail.csv", encoding='ISO-8859-1')

# Basic structure
print(df.info())
print(df.describe())
print(df.isnull().sum())

# Preview unique countries and missing CustomerID
print("\nCountries:", df['Country'].unique())
print("\nMissing Customer IDs:", df['CustomerID'].isna().sum())

# Remove rows with missing CustomerID and cancellations
clean_df = df.dropna(subset=['CustomerID'])
clean_df = clean_df[~clean_df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove negative/zero quantities and prices
clean_df = clean_df[(clean_df['Quantity'] > 0) & (clean_df['UnitPrice'] > 0)]

# Add TotalPrice column
clean_df['TotalPrice'] = clean_df['Quantity'] * clean_df['UnitPrice']
clean_df['InvoiceDate'] = pd.to_datetime(clean_df['InvoiceDate'])

# Save cleaned version if needed
os.makedirs("data", exist_ok=True)
clean_df.to_csv("/content/cleaned_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None
            Quantity      UnitPrice     CustomerID
count  541909.000000  541909.000000  406829.000000
mean        9.552250       4.611114   15287.690570
std       218.081158      96.759853    1713.600303
min    -80995.000000  -11062.060000   12346.000000
25%         1.000000       1.250000   13953.000000
50%         3.000000       2.080000   15152.000000
75%        10.000000       4.130000

data is cleaned and new data csv is checked here

In [7]:
df = pd.read_csv("/content/cleaned_data.csv", encoding='ISO-8859-1')

# Basic structure
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397884 entries, 0 to 397883
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397884 non-null  int64  
 1   StockCode    397884 non-null  object 
 2   Description  397884 non-null  object 
 3   Quantity     397884 non-null  int64  
 4   InvoiceDate  397884 non-null  object 
 5   UnitPrice    397884 non-null  float64
 6   CustomerID   397884 non-null  float64
 7   Country      397884 non-null  object 
 8   TotalPrice   397884 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 27.3+ MB
None
           InvoiceNo       Quantity      UnitPrice     CustomerID  \
count  397884.000000  397884.000000  397884.000000  397884.000000   
mean   560616.934451      12.988238       3.116488   15294.423453   
std     13106.117773     179.331775      22.097877    1713.141560   
min    536365.000000       1.000000       0.001000   12346.000000   
25% 

In [11]:
# ============================
# Step 2: Exploratory Analysis
# ============================

# Top selling products
top_products = clean_df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\nTop Products:\n", top_products)

# Transactions by country
tx_by_country = clean_df.groupby('Country')['InvoiceNo'].nunique().sort_values(ascending=False)
print("\nTransactions by Country:\n", tx_by_country.head(10))

# Time trend of purchases
monthly_sales = clean_df.set_index('InvoiceDate')['TotalPrice'].resample('M').sum()
plt.figure(figsize=(10, 4))
monthly_sales.plot(title='Monthly Sales Trend')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.tight_layout()
plt.savefig("/content/data/chartsmonthly_sales.png")
plt.close()



Top Products:
 Description
PAPER CRAFT , LITTLE BIRDIE           80995
MEDIUM CERAMIC TOP STORAGE JAR        77916
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54415
JUMBO BAG RED RETROSPOT               46181
WHITE HANGING HEART T-LIGHT HOLDER    36725
ASSORTED COLOUR BIRD ORNAMENT         35362
PACK OF 72 RETROSPOT CAKE CASES       33693
POPCORN HOLDER                        30931
RABBIT NIGHT LIGHT                    27202
MINI PAINT SET VINTAGE                26076
Name: Quantity, dtype: int64

Transactions by Country:
 Country
United Kingdom    16646
Germany             457
France              389
EIRE                260
Belgium              98
Netherlands          94
Spain                90
Portugal             57
Australia            57
Switzerland          51
Name: InvoiceNo, dtype: int64


/tmp/ipython-input-3143276423.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = clean_df.set_index('InvoiceDate')['TotalPrice'].resample('M').sum()


In [12]:
# =============================
# Step 3: RFM Feature Creation
# =============================

snapshot_date = clean_df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = clean_df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

In [13]:
# =============================
# Step 4: Clustering Evaluation
# =============================

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

# Elbow method
inertia = []
sil_scores = []
for k in range(2, 10):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(rfm_scaled)
    inertia.append(model.inertia_)
    sil_scores.append(silhouette_score(rfm_scaled, model.labels_))

In [17]:
# Plot Elbow Curve
plt.figure(figsize=(6, 4))
plt.plot(range(2, 10), inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('K')
plt.ylabel('Inertia')
plt.tight_layout()
plt.savefig("/content/data/charts/elbow_curve.png")
plt.close()

# Plot Silhouette Score
plt.figure(figsize=(6, 4))
plt.plot(range(2, 10), sil_scores, marker='s', color='green')
plt.title('Silhouette Scores')
plt.xlabel('K')
plt.ylabel('Score')
plt.tight_layout()
plt.savefig("/content/data/charts/silhouette_score.png")
plt.close()

In [18]:
# Best model (based on elbow/silhouette visually decided as 4)
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Cluster interpretation
cluster_summary = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
print("\nCluster Summary:\n", cluster_summary)

# Save models for Streamlit
os.makedirs("app/model", exist_ok=True)
pickle.dump(kmeans, open("/content/data/model/kmeans_model.pkl", "wb"))
pickle.dump(scaler, open("/content/data/model/scaler.pkl", "wb"))


Cluster Summary:
             Recency  Frequency       Monetary
Cluster                                      
0         43.702685   3.682711    1359.049284
1        248.075914   1.552015     480.617480
2          7.384615  82.538462  127338.313846
3         15.500000  22.333333   12709.090490


In [20]:
# =============================
# Step 5: Product Similarity Matrix
# =============================
from sklearn.metrics.pairwise import cosine_similarity

pivot = clean_df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum', fill_value=0)
sim_matrix = cosine_similarity(pivot.T)
sim_df = pd.DataFrame(sim_matrix, index=pivot.columns, columns=pivot.columns)
sim_df.to_csv("data/similarity_matrix.csv")

In [21]:
# =============================
# Step 6: Save RFM & Clustered Data
# =============================

rfm.to_csv("data/rfm_with_clusters.csv", index=False)

print("\n✅ Analysis Complete. All outputs and models saved.")


✅ Analysis Complete. All outputs and models saved.
